# Gaussian Process and Neural Network Gaussian Processes

This notebook is for looking at traditional gaussian processes and their neural network implementation for use in optimal control and state prediction. Various parameters are test on a real dataset

## Import libraries and Dataset

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import gaussian_process
from thesis.data import blackbird_dataset as rbd

In [2]:
# Generate Test Dataset
test_df = rbd.cleaned_blackbird_test(
    'figure8', 'Constant', 0.5
)

In [5]:
# Set up downsampling
downsample_dict={
        'stride_pos': 36,          # 10 Hz
        'stride_att': 36,          # 10 Hz
        'stride_pos_ref': 19,      # 10 Hz
        'stride_att_ref': 19,      # 10 Hz
        'stride_motor_speeds': 19, # 10 Hz
        'stride_accel': 10,        # 10 Hz
        'stride_gyro': 10,         # 10 Hz
        'stride_pwms': 10,           # 20 Hz
    }

# Get optimal control input/output
X, Y, tvec_y, info = rbd.generate_opt_control_test_data(
    test_df,
    past_delta_t=2,
    future_delta_t=1,
    downsample_dict=downsample_dict
)

for ko, kd in zip(info.keys(), downsample_dict.keys()):
    n, dt = info[ko]
    d = downsample_dict[kd]
    print("\t%s: %i [samples]\t%f [Hz]" %(ko, int(np.ceil(n/d)), 1./(dt*d)))

	vicon position: 20 [samples]	9.999200 [Hz]
	vicon attitude: 20 [samples]	9.999200 [Hz]
	reference position: 10 [samples]	9.919257 [Hz]
	reference attitude: 10 [samples]	9.919257 [Hz]
	motor speeds: 20 [samples]	9.889436 [Hz]
	accelerometer: 20 [samples]	10.000000 [Hz]
	gyroscope: 20 [samples]	10.000000 [Hz]
	PWM: 21 [samples]	20.881622 [Hz]


In [6]:
# Describing the data
print("Number of test points: %i" % X.shape[0])
print("Input dimension: %i" % X.shape[1])
print("Output dimension: %s" % Y.shape[1:])

Number of test points: 38089
Input dimension: 410
Output dimension: 84


## Gaussian Processes

Gaussian processes are fairly well known and established. 

$$p(\vec{x}) \sim \mathcal{N}\left(\vec{\mu}, \Sigma \right) = \frac{1}{\sqrt{\left(2\pi\right)^2 \left|\Sigma\right|}}exp\left(-\frac{1}{2}\vec{x}^T \Sigma^{-1} \vec{x}\right)$$

$$p(\vec{x}, \vec{y}) \sim \mathcal{N}\left(\begin{bmatrix}\mu_x\\ \mu_y\end{bmatrix}, \begin{bmatrix}\Sigma_{xx}&\Sigma_{xy}\\\Sigma_{yx}&\Sigma_{yy}\end{bmatrix}\right)$$

$$p(\vec{x}|\vec{y}) \sim \mathcal{N} \left(\mu_x + \Sigma_{xy}\Sigma_{yy}^{-1}(\vec{y} - \mu_y), \Sigma_{xx} - \Sigma_{xy}\Sigma_{yy}^{-1}\Sigma_{xy}^T\right)$$

$$p(\vec{x}) \sim \mathcal{GP}\left(\mu(\vec{x}), k(\vec{x},\vec{x}')\right)$$

## Neural Network Gaussian Processes